<a href="https://colab.research.google.com/github/AdnaneMOUZAOUI/Analyse-de-sentiments-machine-learning/blob/Reviews-Analysis/Pr%C3%A9paration_extraction_des_commentaires_uniquement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importer le fichier depuis le drive
import pandas as pd

df = pd.read_csv('showroom_trustpilot_168897_07 mars.csv')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk
import re
from sklearn.feature_selection import VarianceThreshold, SelectKBest, SelectFromModel, f_regression, mutual_info_regression, RFE, RFECV
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

In [ ]:
df=df.rename(columns={"nom_client":"client","date_achat":"date_commande","date_avis":"date","text_avis":"Commentaire","titre_avis":"Titre","note_avis":"star"})

In [ ]:
# Comptage et transformation des commentaires

from collections import Counter
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import ngrams
from tqdm import tqdm

lemmatizer = WordNetLemmatizer()

commentaires = pd.Series(df['Commentaire'])
stop_words = set(stopwords.words('french'))
stop_words.update(['a', "jai", "cest", "sest", "car", "donc"])

def clean_text(text):
    text = " ".join(text.split()) # supprimer les espaces superflus
    text = text.lower() # mettre les mots en minuscule
    text = text.translate(str.maketrans('', '', string.punctuation)) # retirer les ponctuations
    return text

def count_words(text):
    return len(text.split())

# Calculer la longueur de chaque commentaire
df['longueur'] = commentaires.apply(lambda x: len(x) if isinstance(x, str) else 0)

# Calculer le nombre de majuscules dans chaque commentaire
df['majuscule'] = commentaires.apply(lambda x: sum(1 for c in x if c.isupper()) if isinstance(x, str) else 0)

# Calculer le nombre de points d'exclamation et d'interrogation dans chaque commentaire
df['ponct'] = commentaires.apply(lambda x: x.count('!') + x.count('?') if isinstance(x, str) else 0)

# Compter le nombre de mots dans chaque commentaire
df['nb_mots'] = commentaires.apply(lambda x: count_words(x) if isinstance(x, str) else 0)

# Nettoyer le texte de chaque commentaire, lemmatiser et compter la fréquence des mots
for x in tqdm(range(0,df.shape[0])):
    try:
        filtered_sentence = []
        commentaires[x] = clean_text(commentaires[x])
        word_tokens = word_tokenize(commentaires[x], language="french")
        for w in word_tokens:
            if not w in stop_words and w.isalpha():
                lemma = lemmatizer.lemmatize(w)
                filtered_sentence.append(lemma)
        commentaires[x] = filtered_sentence
    except:
        commentaires[x] = ""


df['Commentaire'] = commentaires


# Créer des n-grammes de taille 2 et 3
df['ngram_2'] = commentaires.apply(lambda x: list(ngrams(x, 2)) if isinstance(x, list) else [])
df['ngram_3'] = commentaires.apply(lambda x: list(ngrams(x, 3)) if isinstance(x, list) else [])



100%|████████████████████████████████████████████████████████████████████████| 168897/168897 [00:51<00:00, 3258.95it/s]


20 mots les plus communs :
commande 56221
livraison 48559
très 42398
bien 25659
colis 25619
site 24226
plus 23735
produits 21646
produit 21337
article 18585
qualité 16756
tout 16474
bon 16045
reçu 15087
délais 13681
prix 13328
trop 12925
long 12839
toujours 12055
peu 11873

 20 2-ngrams les plus communs :
('délai', 'livraison') 5899
('très', 'bien') 5658
('délais', 'livraison') 5497
('trop', 'long') 4766
('service', 'client') 4570
('point', 'relais') 4465
('très', 'bon') 4103
('bonne', 'qualité') 3820
('frais', 'port') 3758
('très', 'satisfaite') 3511
('peu', 'long') 3466
('rien', 'dire') 3240
('date', 'lexpérience') 2935
('livraison', 'trop') 2910
('livraison', 'peu') 2832
('bien', 'passé') 2613
('produit', 'conforme') 2447
('bon', 'site') 2418
('cette', 'commande') 2320
('bon', 'produit') 2301

 20 3-ngrams les plus communs :
('livraison', 'trop', 'long') 1587
('livraison', 'peu', 'long') 1364
('très', 'bon', 'site') 1334
('tout', 'bien', 'passé') 1221
('rapport', 'qualité', 'prix') 

# Préparation modélisation

In [ ]:
# Création d'une nouvelle variable "star_cluster" pour séparer le jeu de données en 2 segments,
# le segment des avis positifs (=1) et le segment des avis négatifs (=0)
df['star_cluster'] = df['star'].apply(lambda x: 1 if x >= 4 else 0)

In [ ]:
# réinitialisation des compteurs
compteur_mots_com = Counter()
compteur_ngrams_2_com = Counter()
compteur_ngrams_3_com = Counter()


for x in range(0,df.shape[0]):
    mots = df.loc[x, 'Commentaire']
    ngrams_2 = df.loc[x, 'ngram_2']
    ngrams_3 = df.loc[x, 'ngram_3']
    compteur_mots_com.update(mots)
    compteur_ngrams_2_com.update(ngrams_2)
    compteur_ngrams_3_com.update(ngrams_3)




In [ ]:
# Dichotomisation des Commentaires
df['Commentaire'].fillna('', inplace=True)

mots_cles = [mot for mot, _ in compteur_mots_com.most_common(20)] #on utilise ", _" pour indiquer que l'on veut stocker uniquement le premier élement de 'most_common(), et non pas le compte d'apparition

for mot in mots_cles:
    df['Commentaire_'+mot] = df['Commentaire'].apply(lambda x: int(mot in x)) #int pour convertir le booléen en entier


# Dichotomisation des ngrams2

df['ngram_2'].fillna('', inplace=True)
mots_cles = [mot for mot, _ in compteur_ngrams_2_com.most_common(20)] #on utilise ", _" pour indiquer que l'on veut stocker uniquement le premier élement de 'most_common(), et non pas le compte d'apparition

# Créer une colonne pour chaque tuples clés
for mot in mots_cles:
    df['ngrams_2_'+str(mot)] = df['ngram_2'].apply(lambda x: int(any(mot == t for t in x)))

# Dichotomiser les ngrams2
dummies = pd.get_dummies(df[['ngrams_2_'+str(mot) for mot in mots_cles]])

# Ajouter les colonnes dichotomisées à la dataframe
df = pd.concat([df, dummies], axis=1)


# Dichotomisation des ngrams3
df['ngram_3'].fillna('', inplace=True)
mots_cles = [mot for mot, _ in compteur_ngrams_3_com.most_common(20)] #on utilise ", _" pour indiquer que l'on veut stocker uniquement le premier élement de 'most_common(), et non pas le compte d'apparition

# Créer une colonne pour chaque tuples clés
for mot in mots_cles:
    df['ngrams_3_'+str(mot)] = df['ngram_3'].apply(lambda x: int(any(mot == t for t in x)))

# Dichotomiser les ngrams3
dummies = pd.get_dummies(df[['ngrams_3_'+str(mot) for mot in mots_cles]])

# Ajouter les colonnes dichotomisées à la dataframe
df = pd.concat([df, dummies], axis=1)



C:\Users\f.rouxelin\Anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [ ]:

df.to_csv('commentaires_prepares.csv')